In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [18]:
spark = SparkSession.builder.appName('Aggregate exercise').getOrCreate()
spark

In [51]:
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
schema = ["employee_name", "department", "salary"]
df = spark.createDataFrame(simpleData,schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [20]:
df.createOrReplaceTempView('df_sql')

In [21]:
df_agg = df.agg(approx_count_distinct("salary").alias('Distinct_Salary_count'))
df_agg.show()

+---------------------+
|Distinct_Salary_count|
+---------------------+
|                    6|
+---------------------+



In [22]:
df_agg_sql = spark.sql('select count(distinct(salary)) as Distinct_Salary_count from df_sql').show(truncate=False)

+---------------------+
|Distinct_Salary_count|
+---------------------+
|6                    |
+---------------------+



In [23]:
df_salary_collect_list = df.agg(collect_list('salary').alias('collect_list'))
df_salary_collect_list.show(truncate=False)

+------------------------------------------------------------+
|collect_list                                                |
+------------------------------------------------------------+
|[3000, 4600, 4100, 3000, 3000, 3300, 3900, 3000, 2000, 4100]|
+------------------------------------------------------------+



In [24]:
df_salary_collect_list = spark.sql('select salary as collect_list from df_sql').show(truncate=False)

+------------+
|collect_list|
+------------+
|3000        |
|4600        |
|4100        |
|3000        |
|3000        |
|3300        |
|3900        |
|3000        |
|2000        |
|4100        |
+------------+



In [25]:
df_salary_collect_set = df.agg(collect_set('salary').alias('collect_set'))
df_salary_collect_set.show(truncate=False)

+------------------------------------+
|collect_set                         |
+------------------------------------+
|[4600, 3000, 3900, 4100, 3300, 2000]|
+------------------------------------+



In [26]:
df_salary_collect_list = spark.sql('select distinct(salary) as collect_set from df_sql').show(truncate=False)

+-----------+
|collect_set|
+-----------+
|4600       |
|3000       |
|4100       |
|3300       |
|3900       |
|2000       |
+-----------+



In [27]:
df_count = df.agg(count('salary').alias('count')).show(truncate=False)

+-----+
|count|
+-----+
|10   |
+-----+



In [28]:
df_count_sql = spark.sql('select count(*) as count from df_sql').show(truncate=False)

+-----+
|count|
+-----+
|10   |
+-----+



In [29]:
first_sal = df.agg(first('salary').alias('first_sal')).show(truncate=False)

+---------+
|first_sal|
+---------+
|3000     |
+---------+



In [30]:
last_sal = df.agg(last('salary').alias('last_sal')).show(truncate=False)

+--------+
|last_sal|
+--------+
|4100    |
+--------+



In [31]:
max_sal = df.agg(max('salary').alias('max')).show(truncate=False)

+----+
|max |
+----+
|4600|
+----+



In [32]:
min_sal = df.agg(min('salary').alias('min')).show(truncate=False)

+----+
|min |
+----+
|2000|
+----+



In [33]:
min_sal = df.agg(min('salary')).collect()[0]["min(salary)"]
min_sal

2000

In [34]:
df_filter = df.filter(col('salary') == min_sal)
df_filter.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Kumar| Marketing|  2000|
+-------------+----------+------+



In [36]:
spark.sql('select * from df_sql where salary = (select min(salary) from df_sql)').show(truncate=False)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Kumar        |Marketing |2000  |
+-------------+----------+------+



In [42]:
sum_sal = df.agg(sum('salary').alias('sum_salary'))
sum_sal.show(truncate=False)
sum_sal.printSchema()
sum_sal_sql = spark.sql('select sum(salary) as sum_salary from df_sql').show()

+----------+
|sum_salary|
+----------+
|34000     |
+----------+

root
 |-- sum_salary: long (nullable = true)

+----------+
|sum_salary|
+----------+
|     34000|
+----------+



In [48]:
group_department = df.groupBy('Department').sum('salary')
group_department = group_department.withColumnRenamed('sum(salary)' , 'sum_depart_sal')
group_department.show()

+----------+--------------+
|Department|sum_depart_sal|
+----------+--------------+
|     Sales|         18800|
|   Finance|         10200|
| Marketing|          5000|
+----------+--------------+



In [49]:
group_department_sql = spark.sql('select Department, sum(salary) as sum_depart_sal from df_sql group by 1').show()

+----------+--------------+
|Department|sum_depart_sal|
+----------+--------------+
|     Sales|         18800|
|   Finance|         10200|
| Marketing|          5000|
+----------+--------------+



In [63]:
group_department =  df.groupBy('Department')\
                    .agg( \
                          count(col('salary')).alias('count_sal'), \
                          sum(col('salary')).alias('Total_sal'), \
                          min(col('salary')).alias('min_sal'), \
                          max(col('salary')).alias('max_sal'), \
                          avg(col('salary')).alias('avg_sal') \
                        ).orderBy(col('count_sal').asc())

group_department.show()

+----------+---------+---------+-------+-------+-------+
|Department|count_sal|Total_sal|min_sal|max_sal|avg_sal|
+----------+---------+---------+-------+-------+-------+
| Marketing|        2|     5000|   2000|   3000| 2500.0|
|   Finance|        3|    10200|   3000|   3900| 3400.0|
|     Sales|        5|    18800|   3000|   4600| 3760.0|
+----------+---------+---------+-------+-------+-------+



In [64]:
group_department_sql = spark.sql('select Department, \
                                 count(salary) as count_sal, \
                                 sum(salary) as Total_sal, \
                                 min(salary) as min_sal, \
                                 max(salary) as max_sal, \
                                 avg(salary) as avg_sal \
                                 from df_sql \
                                 group by Department \
                                 order by count_sal\
                                 ')
                                 
group_department_sql.show(truncate=False)                                 

+----------+---------+---------+-------+-------+-------+
|Department|count_sal|Total_sal|min_sal|max_sal|avg_sal|
+----------+---------+---------+-------+-------+-------+
|Marketing |2        |5000     |2000   |3000   |2500.0 |
|Finance   |3        |10200    |3000   |3900   |3400.0 |
|Sales     |5        |18800    |3000   |4600   |3760.0 |
+----------+---------+---------+-------+-------+-------+



In [66]:
# Create DataFrame
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [84]:
df.createOrReplaceTempView('df_sql')

In [78]:
df.groupBy('state', 'department')\
        .agg(\
            sum('salary').alias('Total_Sal'), \
            avg('salary').alias('avg_Sal')).orderBy(col('Total_Sal').desc())\
            .where(col('state') == 'NY').show(truncate=False)

+-----+----------+---------+-------+
|state|department|Total_Sal|avg_Sal|
+-----+----------+---------+-------+
|NY   |Sales     |176000   |88000.0|
|NY   |Finance   |162000   |81000.0|
|NY   |Marketing |91000    |91000.0|
+-----+----------+---------+-------+



In [89]:
spark.sql(' \
            select state, department,\
            sum(salary) as Total_sal,\
            avg(salary) as avg_sal \
            from df_sql \
            group by state, department\
            having state = "NY" \
            ').show()

+-----+----------+---------+-------+
|state|department|Total_sal|avg_sal|
+-----+----------+---------+-------+
|   NY|     Sales|   176000|88000.0|
|   NY|   Finance|   162000|81000.0|
|   NY| Marketing|    91000|91000.0|
+-----+----------+---------+-------+

